In [28]:
import rpyc

In [29]:
c = rpyc.connect("localhost", 18861)
c.root

In [30]:
c.root.perform_multiplication([10, 10])

[100, 100]

In [31]:
c.close()

In [ ]:
from computer_utils import SimpleSplitter, SimpleMerger
from multiprocessing import Queue

class Overseer:
    
    def __init__(self, connection):
        self.c = connection
    
    def start(self, queue, weight, res_dict):
        while not(queue.empty()):
            

class Computer:
    
    def __init__(self, data, weight):
        self.data = data
        self.object_queue = Queue()
        self.weight = weight
        
        self.conns = {}
        
#         self.pinger = SimplePinger(self.conns)
        self.splitter = SimpleSplitter()
        self.merger = SimpleMerger()
        
    def add_connection(self, ip = 'localhost', port = 18861):
        self.conns[f'{ip}:{port}'] = rpyc.connect(ip, port)
        
    def process(self):
        queue = Queue()
        self.splitter.split(self.data, queue)
        while 